<a href="https://colab.research.google.com/github/MengOonLee/Container/blob/main/Notebook/Docker/GPU/build_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile run_notebook.sh
#!/bin/bash

docker run --gpus all --rm -it \
  -p 8888:8888 \
  -p 8050:8050 \
  -v $PWD:/Work \
  -w /Work \
  tensorflow/tensorflow:latest-gpu \
  bash -c "
    python3 -m venv venv;
    source venv/bin/activate;
    pip install --no-cache-dir -U \
      pip wheel build jupyterlab \
      numpy pandas pyarrow scikit-learn \
      apache-beam[interactive] \
      matplotlib seaborn dash \
      tensorflow tensorflow-datasets tensorflow-probability \
      torch torchvision torchaudio;
    pip check;
    ipython kernel install --user --name=venv;
    jupyter-lab \
      --ip=0.0.0.0 \
      --port=8888 \
      --no-browser \
      --allow-root \
      --ContentsManager.allow_hidden=True
  "

In [ ]:
%%writefile fix-permissions
#!/bin/bash

set -e

for d in "$@"; do
    find "${d}" \
        ! \( \
            -group "${NB_GID}" \
            -a -perm -g+rwX \
        \) \
        -exec chgrp "${NB_GID}" {} \; \
        -exec chmod g+rwX {} \;
    # setuid, setgid *on directories only*
    find "${d}" \
        \( \
            -type d \
            -a ! -perm -6000 \
        \) \
        -exec chmod +6000 {} \;
done

Overwriting fix-permissions


In [ ]:
%%writefile Dockerfile
ARG BASE=tensorflow/tensorflow:latest-gpu
FROM $BASE

ARG NB_USER=meng
ARG NB_UID=1000
ARG NB_GID=100

ENV SHELL=/bin/bash
ENV NB_USER=$NB_USER
ENV NB_UID=$NB_UID
ENV NB_GID=$NB_GID
ENV HOME=/home/$NB_USER
ENV VENV=$HOME/venv
ENV PATH=$VENV/bin:$PATH
ENV JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64

COPY fix-permissions /usr/local/bin/fix-permissions

ENV DEBIAN_FRONTEND noninteractive
RUN apt-get update -y \
    && apt-get upgrade -y \
    && apt-get autoremove --purge -y \
    && apt-get install -y \
        python3 \
        python3-venv \
        pandoc \
        texlive-xetex \
        texlive-fonts-recommended \
        texlive-plain-generic \
        texlive-latex-extra \
        texlive-science \
        openjdk-11-jre-headless \
    && pip freeze | xargs pip uninstall -y \
    && apt-get update -y \
    && apt-get upgrade -y \
    && apt-get autoremove --purge -y \
    && apt-get autoclean -y && apt-get clean -y \
    && rm -rf /var/lib/apt/lists/* \
    && update-alternatives --config java \
    && chmod a+rx /usr/local/bin/fix-permissions \
    && useradd -l -m -s /bin/bash -N -u $NB_UID $NB_USER \
    && mkdir -p $HOME \
    && chown $NB_USER:$NB_GID $HOME \
    && chmod g+w /etc/passwd \
    && fix-permissions $HOME

USER $NB_UID

WORKDIR $HOME

RUN python3 -m venv $VENV \
    && python3 -m pip install --no-cache-dir -U pip wheel \
    && pip check \
    && rm -rf .cache

Overwriting Dockerfile


In [ ]:
%%writefile build.sh
#!/bin/bash

export REPOSITORY=darklemon
export IMAGE=tensorflow-gpu
export TAG=latest

docker build . --no-cache --pull \
-f Dockerfile \
-t $REPOSITORY/$IMAGE:$TAG

# docker push $REPOSITORY/$IMAGE:$TAG

Overwriting build.sh


In [ ]:
%%writefile /home/meng/Work/run_tensorflow.sh
#!/bin/bash

docker run --gpus all --rm -it -p 8888:8888 -v $PWD:/home/meng/Work darklemon/tensorflow-gpu:latest

Overwriting /home/meng/Work/run_tensorflow.sh


In [ ]:
%%writefile /home/meng/Work/run_notebook.sh
#!/bin/bash

# pip install --no-cache-dir -U \
#    pip wheel build notebook
pip check
ipython kernel install --user --name=env
jupyter notebook \
    --ip 0.0.0.0 --port 8888 \
    --no-browser --allow-root

Overwriting /home/meng/Work/run_notebook.sh
